In [68]:
pip install trimesh

   ---------------------------------------- 0.0/708.6 kB ? eta -:--:--
   ---------------------------------------- 708.6/708.6 kB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [54]:
import open3d as o3d
import numpy as np

def load_obj_mesh(filename):
    """
    Loads an OBJ file as an Open3D TriangleMesh.
    """
    mesh = o3d.io.read_triangle_mesh(filename)
    if not mesh.has_triangles():
        print("Warning: Mesh does not contain triangles.")
    mesh.compute_vertex_normals()
    return mesh

def segment_planes(pcd, distance_threshold=0.0001, ransac_n=3, num_iterations=1000, min_inliers=100):
    """
    Repeatedly segments the largest plane from the point cloud using RANSAC until too few points remain.
    Returns a list of plane equations and the corresponding plane point clouds.
    """
    planes = []
    plane_models = []
    remaining_pcd = pcd
    while True:
        if len(remaining_pcd.points) < min_inliers:
            break
        plane_model, inliers = remaining_pcd.segment_plane(distance_threshold=distance_threshold,
                                                           ransac_n=ransac_n,
                                                           num_iterations=num_iterations)
        if len(inliers) < min_inliers:
            break
        plane_models.append(plane_model)
        plane_cloud = remaining_pcd.select_by_index(inliers)
        planes.append(plane_cloud)
        remaining_pcd = remaining_pcd.select_by_index(inliers, invert=True)
    return plane_models, planes

def classify_planes(plane_models, angle_threshold=15):
    """
    Classify each plane by comparing its normal to the vertical (z-axis).
    Planes with normals within angle_threshold degrees of vertical (or its inverse) are treated as floors/ceilings;
    those near 90° are considered walls.
    """
    floor_planes = []
    wall_planes = []
    for plane in plane_models:
        a, b, c, d = plane
        normal = np.array([a, b, c])
        normal = normal / np.linalg.norm(normal)
        vertical = np.array([0, 0, 1])
        angle_deg = np.degrees(np.arccos(np.clip(np.dot(normal, vertical), -1.0, 1.0)))
        if angle_deg < angle_threshold or angle_deg > (180 - angle_threshold):
            floor_planes.append(plane)
        elif abs(angle_deg - 90) < angle_threshold:
            wall_planes.append(plane)
    return floor_planes, wall_planes

def adjust_floor_levels(floor_planes):
    """
    Provides a simple interactive prompt to adjust detected floor heights.
    For horizontal planes (floors), the plane equation is assumed to be in the form ax + by + cz + d = 0.
    Solving for z gives z = -d/c.
    """
    new_floor_planes = []
    print("Detected floor levels:")
    for i, plane in enumerate(floor_planes):
        a, b, c, d = plane
        if abs(c) > 1e-6:
            floor_height = -d / c
        else:
            floor_height = None
        print(f"Floor {i+1}: approximate height = {floor_height}")
        user_input = input(f"Enter adjusted height for floor {i+1} (or press Enter to keep {floor_height}): ")
        if user_input:
            try:
                new_height = float(user_input)
            except ValueError:
                new_height = floor_height
        else:
            new_height = floor_height
        new_d = -new_height * c if c != 0 else d
        new_plane = (a, b, c, new_d)
        new_floor_planes.append(new_plane)
    return new_floor_planes

def main():
    # Replace 'yourfile.obj' with the path to your OBJ file.
    filename = r"D:\IaaC\RESEARCH\GITHUB\Octopusie\sahils experiments\Reference files\3d model for detection.obj"
    mesh_o3d = load_obj_mesh(filename)
    print("Mesh loaded. Number of vertices:", len(mesh_o3d.vertices))
    
    # If the mesh has triangles, sample a point cloud from the mesh.
    if mesh_o3d.has_triangles():
        pcd = mesh_o3d.sample_points_uniformly(number_of_points=100000)
    else:
        print("Mesh does not have triangles; converting mesh vertices directly to a point cloud.")
        pcd = o3d.geometry.PointCloud()
        pcd.points = mesh_o3d.vertices
    
    # Segment planes using RANSAC.
    plane_models, plane_clouds = segment_planes(pcd)
    print(f"Found {len(plane_models)} planes in the geometry.")
    
    # Classify segmented planes into floors and walls.
    floor_planes, wall_planes = classify_planes(plane_models)
    print(f"Detected {len(floor_planes)} floor/ceiling planes and {len(wall_planes)} wall planes.")
    
    # Allow the user to input the number of floors and check the floor-to-floor height.
    while True:
        num_floors = int(input("Enter the number of floors: "))
        if len(floor_planes) < 2:
            print("Not enough floor planes detected to calculate floor-to-floor height.")
            break
        floor_heights = sorted([-plane[3] / plane[2] for plane in floor_planes if abs(plane[2]) > 1e-6])
        if len(floor_heights) < 2:
            print("Not enough valid floor heights detected.")
            break
        min_height = min(floor_heights)
        max_height = max(floor_heights)
        floor_to_floor_height = (max_height - min_height) / (num_floors - 1)
        if floor_to_floor_height < 3:
            print(f"Floor-to-floor height of {floor_to_floor_height:.2f} is less than 3. Please enter a smaller number of floors.")
        else:
            print(f"Floor-to-floor height is {floor_to_floor_height:.2f}.")
            break
    
    # Allow the user to adjust the detected floor levels.
    adjusted_floor_planes = adjust_floor_levels(floor_planes)
    print("Adjusted Floor Planes:")
    for plane in adjusted_floor_planes:
        print(plane)
    
    # Visualize the point cloud along with all segmented planes.
    o3d.visualization.draw_geometries([pcd] + plane_clouds)

if __name__ == "__main__":
    main()


Mesh loaded. Number of vertices: 42
Found 9 planes in the geometry.
Detected 2 floor/ceiling planes and 3 wall planes.
Floor-to-floor height is 22.98.
Detected floor levels:
Floor 1: approximate height = 113.84996032714844
Floor 2: approximate height = 21.92091941833496
Adjusted Floor Planes:
(np.float64(9.584146930523242e-19), np.float64(9.233380126179731e-18), np.float64(1.0), np.float64(-5.5))
(np.float64(4.185928098947046e-19), np.float64(-6.157774467043687e-19), np.float64(1.0), np.float64(-5.0))
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [ ]:
import rhino3dm
import open3d as o3d
import numpy as np

def load_rhino_mesh(filename):
    """
    Loads a Rhino 3DM file, converts Brep geometries to meshes,
    and returns combined vertices and triangle indices.
    """
    model = rhino3dm.File3dm.Read(filename)
    if model is None:
        print(f"Error: Could not load file: {filename}")
        return None, None

    all_vertices = []
    all_triangles = []
    vertex_offset = 0

    # Iterate over all objects in the file
    for obj in model.Objects:
        geom = obj.Geometry
        if isinstance(geom, rhino3dm.Brep):
            # Convert NURBS Brep to mesh with default meshing parameters
            mesh = geom.GetMesh(rhino3dm.MeshType.Any)
            if mesh and mesh.Vertices.Count > 0 and mesh.Faces.Count > 0:
                # Extract vertices from the mesh
                vertices = np.array([[mesh.Vertices[i].X, mesh.Vertices[i].Y, mesh.Vertices[i].Z]
                                     for i in range(mesh.Vertices.Count)])
                faces = []
                # Rhino faces can be triangles or quads.
                # If quad, split it into two triangles.
                for i in range(mesh.Faces.Count):
                    face = mesh.Faces[i]
                    if face.IsTriangle:
                        faces.append([face.A, face.B, face.C])
                    else:
                        faces.append([face.A, face.B, face.C])
                        faces.append([face.A, face.C, face.D])
                faces = np.array(faces)
                all_vertices.append(vertices)
                all_triangles.append(faces + vertex_offset)
                vertex_offset += vertices.shape[0]

    if all_vertices:
        vertices = np.vstack(all_vertices)
        triangles = np.vstack(all_triangles)
        return vertices, triangles
    else:
        print("No valid mesh data found in the file.")
        return None, None

def convert_to_open3d_mesh(vertices, triangles):
    """
    Converts vertices and triangle indices to an Open3D TriangleMesh.
    """
    mesh_o3d = o3d.geometry.TriangleMesh()
    mesh_o3d.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_o3d.triangles = o3d.utility.Vector3iVector(triangles)
    mesh_o3d.compute_vertex_normals()
    return mesh_o3d

def main():
    # Replace with the path to your Rhino 3DM file
    filename = r"D:\IaaC\RESEARCH\GITHUB\Octopusie\sahils experiments\Reference files\3D MODEL for research.3dm"
    vertices, triangles = load_rhino_mesh(filename)
    if vertices is None:
        return

    mesh_o3d = convert_to_open3d_mesh(vertices, triangles)
    print("Mesh loaded. Number of vertices:", len(mesh_o3d.vertices))
    
    # Visualize the mesh using Open3D
    o3d.visualization.draw_geometries([mesh_o3d])

if __name__ == "__main__":
    main()


No valid mesh data found in the file.


In [53]:
print(geometry)

In [58]:
pip install Axes3D

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement Axes3D (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for Axes3D


In [63]:
pip install matplotlib

  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.1 MB 3.7 MB/s eta 0:00:02
   ------ --------------------------------- 1.3/8.1 MB 3.0 MB/s eta 0:00:03
   --------- ------------------------------ 1.8/8.1 MB 3.0 MB/s eta 0:00:03
   ----------- ---------------------------- 2.4/8.1 MB 2.7 MB/s eta 0:00:03
   -------------- ------------------------- 2.9/8.1 MB 2.8 MB/s eta 0:00:02
   ---------------- ----------------------- 3.4/8.1 MB 2.8 MB/s eta 0:00:02
   ------------------- -------------------- 3.9/8.1 MB 2.7 MB/s eta 0:00:02
   ----------------------- ---------------- 4.7/8.1 MB 2

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [64]:
pip install mpl_toolkits

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement mpl_toolkits (from versions: none)
ERROR: No matching distribution found for mpl_toolkits


In [67]:
import rhino3dm
import numpy as np
import trimesh
import os
import math
from sklearn.cluster import DBSCAN
import argparse

def convert_3dm_to_obj(input_file, output_file):
    """
    Convert a Rhino 3DM file to OBJ format
    """
    print(f"Converting {input_file} to {output_file}...")
    
    # Load the 3DM file
    model = rhino3dm.File3dm.Read(input_file)
    
    # Create a new Trimesh scene to hold all the objects
    scene = trimesh.Scene()
    
    # Process all objects in the 3DM file
    for obj in model.Objects:
        geometry = obj.Geometry
        
        if isinstance(geometry, rhino3dm.Mesh):
            # Convert Rhino mesh to Trimesh format
            vertices = []
            for i in range(geometry.Vertices.Count):
                vertex = geometry.Vertices[i]
                vertices.append([vertex.X, vertex.Y, vertex.Z])
            
            faces = []
            for i in range(geometry.Faces.Count):
                face = geometry.Faces[i]
                if face.IsQuad:
                    # Convert quad to two triangles
                    faces.append([face.A, face.B, face.C])
                    faces.append([face.A, face.C, face.D])
                else:
                    faces.append([face.A, face.B, face.C])
            
            # Create trimesh object
            mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
            scene.add_geometry(mesh)
        
        elif isinstance(geometry, rhino3dm.Brep):
            # Convert Brep to mesh
            mesh = geometry.GetMesh(rhino3dm.MeshType.Render)
            
            if mesh:
                vertices = []
                for i in range(mesh.Vertices.Count):
                    vertex = mesh.Vertices[i]
                    vertices.append([vertex.X, vertex.Y, vertex.Z])
                
                faces = []
                for i in range(mesh.Faces.Count):
                    face = mesh.Faces[i]
                    if face.IsQuad:
                        faces.append([face.A, face.B, face.C])
                        faces.append([face.A, face.C, face.D])
                    else:
                        faces.append([face.A, face.B, face.C])
                
                # Create trimesh object
                tmesh = trimesh.Trimesh(vertices=vertices, faces=faces)
                scene.add_geometry(tmesh)
    
    # Export the scene to OBJ
    scene.export(output_file)
    print(f"Conversion complete. OBJ file saved to: {output_file}")
    
    return output_file

def detect_floor_wall_surfaces(obj_file, num_floors=None, min_floor_height=3.0):
    """
    Analyze an OBJ file to detect which surfaces are floors and which are walls
    based on their orientation (horizontal or vertical)
    
    Parameters:
    obj_file: Path to the OBJ file
    num_floors: Number of floor divisions needed (if None, detect all floors)
    min_floor_height: Minimum acceptable floor-to-floor height
    
    Returns:
    dict: Contains floor and wall meshes, floor heights, and other metrics
    """
    print(f"Analyzing {obj_file} for floor and wall detection...")
    
    # Load the mesh from OBJ file
    mesh = trimesh.load(obj_file)
    
    # Convert to scene if not already
    if isinstance(mesh, trimesh.Trimesh):
        scene = trimesh.Scene([mesh])
    else:
        scene = mesh
    
    # Extract all mesh faces and their normals
    all_faces = []
    all_normals = []
    all_face_centers = []
    
    for name, submesh in scene.geometry.items():
        if isinstance(submesh, trimesh.Trimesh):
            # Get face normals
            normals = submesh.face_normals
            
            # Get face centers
            centers = submesh.triangles_center
            
            # Add to our lists
            for i in range(len(normals)):
                all_normals.append(normals[i])
                all_face_centers.append(centers[i])
                all_faces.append((name, i))  # Store mesh name and face index
    
    # Convert to numpy arrays
    all_normals = np.array(all_normals)
    all_face_centers = np.array(all_face_centers)
    
    # Identify horizontal and vertical faces based on normal vectors
    # For horizontal surfaces (floors/ceilings), the normal is close to (0,0,1) or (0,0,-1)
    vertical_threshold = 0.1  # Threshold for considering a normal as vertical
    horizontal_threshold = 0.9  # Threshold for considering a normal as horizontal
    
    floor_indices = []
    ceiling_indices = []
    wall_indices = []
    
    for i, normal in enumerate(all_normals):
        # Normalize the normal vector
        normalized = normal / np.linalg.norm(normal)
        
        # Check if it's horizontal (floor or ceiling)
        if abs(normalized[2]) > horizontal_threshold:
            if normalized[2] > 0:
                floor_indices.append(i)  # Normal points up (floor)
            else:
                ceiling_indices.append(i)  # Normal points down (ceiling)
        # Check if it's vertical (wall)
        elif abs(normalized[2]) < vertical_threshold:
            wall_indices.append(i)
    
    # Get floor heights (Z coordinates)
    floor_heights = all_face_centers[floor_indices][:, 2] if floor_indices else []
    
    # Cluster floor heights to identify distinct floors
    if len(floor_heights) > 0:
        # Reshape for DBSCAN
        heights = floor_heights.reshape(-1, 1)
        
        # Use DBSCAN for clustering heights
        eps = 0.1  # Maximum distance between two samples for them to be in the same cluster
        db = DBSCAN(eps=eps, min_samples=2).fit(heights)
        
        labels = db.labels_
        unique_labels = set(labels)
        
        # Get the average height for each cluster
        clustered_floors = {}
        for label in unique_labels:
            if label != -1:  # Skip noise points
                cluster_indices = np.where(labels == label)[0]
                cluster_heights = heights[cluster_indices]
                avg_height = np.mean(cluster_heights)
                clustered_floors[avg_height] = [floor_indices[i] for i in cluster_indices]
        
        # Sort floors by height
        sorted_floor_heights = sorted(clustered_floors.keys())
        
        # Check floor-to-floor heights
        if len(sorted_floor_heights) > 1:
            floor_to_floor_heights = []
            for i in range(1, len(sorted_floor_heights)):
                height_diff = sorted_floor_heights[i] - sorted_floor_heights[i-1]
                floor_to_floor_heights.append(height_diff)
                
                if height_diff < min_floor_height:
                    print(f"WARNING: Floor-to-floor height between levels at {sorted_floor_heights[i-1]:.2f} and {sorted_floor_heights[i]:.2f} is {height_diff:.2f}m, which is less than the recommended minimum of {min_floor_height}m.")
        
        # Process number of floors if specified
        if num_floors is not None and len(sorted_floor_heights) != num_floors:
            print(f"NOTE: Detected {len(sorted_floor_heights)} floors, but user requested {num_floors} floors.")
            
            if len(sorted_floor_heights) > num_floors:
                print("You may need to merge some floors.")
            else:
                print("You may need to add more floor divisions.")
                
                # Calculate suggested floor heights for the requested number of floors
                building_height = sorted_floor_heights[-1] - sorted_floor_heights[0]
                suggested_floor_height = building_height / (num_floors - 1)
                
                print(f"For {num_floors} evenly spaced floors, the floor-to-floor height would be approximately {suggested_floor_height:.2f}m.")
    else:
        print("No floor surfaces detected in the model.")
        sorted_floor_heights = []
        clustered_floors = {}
    
    # Create result dictionary
    result = {
        "floor_indices": floor_indices,
        "ceiling_indices": ceiling_indices,
        "wall_indices": wall_indices,
        "floor_heights": sorted_floor_heights,
        "floor_clusters": clustered_floors,
        "face_centers": all_face_centers,
        "face_references": all_faces
    }
    
    # Print summary
    print(f"Analysis complete. Detected {len(sorted_floor_heights)} distinct floor levels and {len(wall_indices)} wall surfaces.")
    
    return result

def visualize_results(obj_file, analysis_result, output_dir):
    """
    Create visualization of the detected floors and walls
    """
    # Load the original mesh
    mesh = trimesh.load(obj_file)
    
    # Convert to scene if not already
    if isinstance(mesh, trimesh.Trimesh):
        scene = trimesh.Scene([mesh])
    else:
        scene = mesh
    
    # Create floor and wall meshes
    floor_meshes = trimesh.Scene()
    wall_meshes = trimesh.Scene()
    
    for name, submesh in scene.geometry.items():
        if isinstance(submesh, trimesh.Trimesh):
            # Create copies for floors and walls
            floor_mesh = submesh.copy()
            wall_mesh = submesh.copy()
            
            # Get indices to keep
            floor_faces = []
            wall_faces = []
            
            for i, (mesh_name, face_idx) in enumerate(analysis_result["face_references"]):
                if mesh_name == name:
                    if i in analysis_result["floor_indices"]:
                        floor_faces.append(face_idx)
                    elif i in analysis_result["wall_indices"]:
                        wall_faces.append(face_idx)
            
            # Keep only floors in floor_mesh
            if floor_faces:
                floor_mesh.update_faces(floor_faces)
                floor_meshes.add_geometry(floor_mesh, node_name=f"{name}_floors")
            
            # Keep only walls in wall_mesh
            if wall_faces:
                wall_mesh.update_faces(wall_faces)
                wall_meshes.add_geometry(wall_mesh, node_name=f"{name}_walls")
    
    # Save the visualizations
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    floor_file = os.path.join(output_dir, "floors.obj")
    wall_file = os.path.join(output_dir, "walls.obj")
    
    floor_meshes.export(floor_file)
    wall_meshes.export(wall_file)
    
    print(f"Visualization files saved to:")
    print(f"  Floors: {floor_file}")
    print(f"  Walls: {wall_file}")

def main():
    # Parse command line arguments
    parser = argparse.ArgumentParser(description='Convert Rhino 3DM to OBJ and analyze floors/walls')
    parser.add_argument('input_file', help='Input Rhino 3DM file')
    parser.add_argument('--output_dir', default='output', help='Output directory for results')
    parser.add_argument('--num_floors', type=int, help='Number of floor divisions needed')
    parser.add_argument('--min_floor_height', type=float, default=3.0, help='Minimum acceptable floor-to-floor height')
    
    args = parser.parse_args(['path/to/your/input.3dm'])
    
    # Create output directory if it doesn't exist
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    
    # Convert 3DM to OBJ
    obj_file = os.path.join(args.output_dir, os.path.splitext(os.path.basename(args.input_file))[0] + '.obj')
    convert_3dm_to_obj(args.input_file, obj_file)
    
    # Analyze the OBJ file
    analysis_result = detect_floor_wall_surfaces(obj_file, args.num_floors, args.min_floor_height)
    
    # Visualize results
    visualize_results(obj_file, analysis_result, args.output_dir)
    
    # Print floor heights
    if analysis_result["floor_heights"]:
        print("\nDetected floor heights:")
        for i, height in enumerate(analysis_result["floor_heights"]):
            print(f"  Floor {i+1}: {height:.2f}m")
    
    print("\nAnalysis complete!")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'trimesh'